In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
key=6361401891

def translate(res,s):
    from googletrans import Translator
    translator = Translator(service_urls=['translate.google.com'])
    if s=="search":
        for jsonstr in res:
            jsonstr['name']=translator.translate(jsonstr['name'], dest="kn").text
    elif s=="crop":
        for jsonstr in res:
            # for i in range(len(jsonstr['crop'])):
            #     jsonstr['crop'][i] = translator.translate(jsonstr['crop'][i], dest="kn").text
            jsonstr['comment']=translator.translate(jsonstr['comment'], dest="kn").text
    return res

@app.route('/cropyield', methods=['get'])
def crop_yield():
    import joblib
    from datetime import date
    import pandas as pd
    model = joblib.load('/home/kisanmitra/mysite/cropYield.pkl')
    district = str(request.args.get('district'))
    year = date.today().year
    season = str(request.args.get('season'))
    crop = str(request.args.get('crop'))
    apikey = int(request.args.get('apikey'))

    if(apikey!=key):
        return jsonify({"error":"Invalid API Key"})

    column_names=['Crop_Year', 'District_Name_BAGALKOT', 'District_Name_BANGALORE RURAL',
       'District_Name_BELGAUM', 'District_Name_BELLARY',
       'District_Name_BENGALURU URBAN', 'District_Name_BIDAR',
       'District_Name_BIJAPUR', 'District_Name_CHAMARAJANAGAR',
       'District_Name_CHIKBALLAPUR', 'District_Name_CHIKMAGALUR',
       'District_Name_CHITRADURGA', 'District_Name_DAKSHIN KANNAD',
       'District_Name_DAVANGERE', 'District_Name_DHARWAD',
       'District_Name_GADAG', 'District_Name_GULBARGA', 'District_Name_HASSAN',
       'District_Name_HAVERI', 'District_Name_KODAGU', 'District_Name_KOLAR',
       'District_Name_KOPPAL', 'District_Name_MANDYA', 'District_Name_MYSORE',
       'District_Name_RAICHUR', 'District_Name_RAMANAGARA',
       'District_Name_SHIMOGA', 'District_Name_TUMKUR', 'District_Name_UDUPI',
       'District_Name_UTTAR KANNAD', 'District_Name_YADGIR', 'Season_Kharif',
       'Season_Rabi', 'Season_Summer', 'Season_Whole Year',
       'Crop_Arcanut (Processed)', 'Crop_Arecanut', 'Crop_Arhar/Tur',
       'Crop_Atcanut (Raw)', 'Crop_Bajra', 'Crop_Banana',
       'Crop_Beans & Mutter(Vegetable)', 'Crop_Black pepper', 'Crop_Brinjal',
       'Crop_Cardamom', 'Crop_Cashewnut', 'Crop_Cashewnut Processed',
       'Crop_Cashewnut Raw', 'Crop_Castor seed', 'Crop_Citrus Fruit',
       'Crop_Coconut', 'Crop_Coriander', 'Crop_Cotton(lint)',
       'Crop_Cowpea(Lobia)', 'Crop_Dry chillies', 'Crop_Dry ginger',
       'Crop_Garlic', 'Crop_Gram', 'Crop_Grapes', 'Crop_Groundnut',
       'Crop_Horse-gram', 'Crop_Jowar', 'Crop_Linseed', 'Crop_Maize',
       'Crop_Mango', 'Crop_Mesta', 'Crop_Moong(Green Gram)', 'Crop_Niger seed',
       'Crop_Onion', 'Crop_Other  Rabi pulses', 'Crop_Other Fresh Fruits',
       'Crop_Other Kharif pulses', 'Crop_Paddy', 'Crop_Papaya',
       'Crop_Peas & beans (Pulses)', 'Crop_Pome Fruit', 'Crop_Potato',
       'Crop_Ragi', 'Crop_Rapeseed &Mustard', 'Crop_Rice', 'Crop_Safflower',
       'Crop_Sannhamp', 'Crop_Sesamum', 'Crop_Small millets', 'Crop_Soyabean',
       'Crop_Sugarcane', 'Crop_Sunflower', 'Crop_Sweet potato', 'Crop_Tapioca',
       'Crop_Tobacco', 'Crop_Tomato', 'Crop_Turmeric', 'Crop_Urad',
       'Crop_Wheat']

    df = pd.DataFrame(columns = column_names)
    df = df.append({'Crop_Year': year, ('District_Name_'+district.upper()): 1, ('Season_'+season): 1, ('Crop_'+crop):1}, ignore_index = True)
    df = df.fillna(0)

    pred=model.predict(df)
    response = jsonify({'prediction':str(pred[0]*10)})
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/crop', methods=['get'])
def crop():
    import joblib
    import pandas as pd

    model = joblib.load('/home/kisanmitra/mysite/crop.pkl')
    n = float(request.args.get('n'))
    p = float(request.args.get('p'))
    k = float(request.args.get('k'))
    temp = float(request.args.get('temp'))
    humidity = float(request.args.get('humidity'))
    ph = float(request.args.get('ph'))
    rainfall = float(request.args.get('rainfall'))
    lang = str(request.args.get('lang'))
    apikey = int(request.args.get('apikey'))

    if(apikey!=key):
        return jsonify({"error":"Invalid API Key"})

    column_names= ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
    df = pd.DataFrame(columns = column_names)

    for i in range(8,14):
        df = df.append({'N': n, 'P': p, 'K': k, 'temperature': temp, 'humidity': humidity, 'ph': ph, 'rainfall': rainfall*(i/10)}, ignore_index = True)

    pred=model.predict(df)

    res=[{'crop':set(), "comment":"- it is the most appropriate crop for the given geographic conditions."},{'crop':set(), 'comment': '- can sustain even when the rainfall is lower than average.'}, {'crop':set(), 'comment': '- does well if with irrigation and water storage facilities.'}]

    for i in range(len(pred)):
        if len(set(pred))==1:
            res[0]["crop"].add(pred[i])
            break
        elif i<len(pred)//2:
            res[1]["crop"].add(pred[i])
        elif i>=len(pred)//2 and pred[i] not in res[1]['crop']:
            res[2]["crop"].add(pred[i])

    res = [x for x in res if len(x["crop"])!=0]
    for x in res:
        x["crop"]=list(x["crop"])

    if lang=="kn":
        res=translate(res,"crop")

    response = jsonify({"prediction":str(res)})
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/search', methods=['get'])
def search_recommendation():
    import joblib
    import pandas as pd
    Id = str(request.args.get('id'))
    lang = str(request.args.get('lang'))
    df = pd.read_csv('/home/kisanmitra/mysite/products.csv')
    apikey = int(request.args.get('apikey'))

    if(apikey!=key):
        return jsonify({"error":"Invalid API Key"})

    with open('/home/kisanmitra/mysite/cosine.data', 'rb') as f:
        cs=joblib.load(f)

    product_index=[]
    for i in Id.split(","):
        product_index.append(df[df.id == i].index[0])
    total = list(map(sum, zip(*[cs[i] for i in product_index])))
    scores = list(enumerate(total))

    sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)

    j=0
    res=[]
    for i in sorted_scores:
        if not i[0] in product_index:
            p = {}
            p["name"] = df['name'][i[0]]
            p["id"] = df['id'][i[0]]
            p["image"] = df['image'][i[0]]
            res.append(p)
        if len(res)>4:
            break
        j+=1

    if lang == "kn":
        res = translate(res,"search")

    response = jsonify(res)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

if __name__ == '__main__':
    app.run()